In [1]:
import os
import json
import random
import pandas as pd
from together import Together

Khởi tạo API và tạo prompt cho LLM phân tích ngữ cảnh gây cười

In [2]:
# Khởi tạo client của Together API
client = Together(api_key="")

# Đường dẫn thư mục
base_dir = "D:\\DS310-P11-Nhom15"
conversation_dir = os.path.join(base_dir, "data", "Videos_text_represenations")

# Hàm load các đoạn hội thoại
def load_conversations():
    data = []
    for i in range(1, 201):
        # Đọc file JSON
        conversation_file = os.path.join(conversation_dir, f"output_{i}.json")
        with open(conversation_file, "r", encoding="utf-8") as f:
            conversation_segments = json.load(f)
        
        data.append({
            "conversation_id": i,
            "conversation_segments": conversation_segments
        })
    return data

# Hàm tạo prompt
def create_prompt(conversation):
    zero_shot_prompt = f"""
Bạn là chuyên gia phân tích và giải thích các tình huống hài hước trong các hội thoại. Tôi sẽ cung cấp các đoạn hội thoại, mỗi đoạn hội thoại được chia thành các segment, mỗi segment bao gồm:
* Utterance: Nội dung lời nói.
* Acoustic Features: Thông tin về giọng nói, tốc độ nói, và cách phát âm.
* Visual Description: Mô tả hình ảnh hoặc ngữ cảnh xuất hiện trong cảnh quay.
Hãy đọc đoạn hội thoại dưới đây và giải thích lý do hài hước trong một đoạn văn ngắn, không vượt quá 100 từ bằng tiếng Việt.

### Đoạn hội thoại:
Hội thoại:
{json.dumps(conversation['conversation_segments'], ensure_ascii=False, indent=2)}
Lý do hài hước:
"""
    return zero_shot_prompt

Cung cấp prompt cho Together API

In [3]:
# Hàm gửi prompt tới Together API
def analyze_conversation(prompt):
    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "user", "content": prompt}],
    )
    print( response.usage.total_tokens)
    return response.choices[0].message.content.strip()

Chọn 2 mẫu random làm ví dụ

In [4]:
# Load dữ liệu
data = load_conversations()

# Chọn 2 mẫu bất kỳ làm ví dụ
examples = random.sample(data, 2)
example_1, example_2 = examples

# Phân tích các mẫu còn lại
results = []
for conversation in data:
    if conversation in examples:
        continue  # Bỏ qua 2 mẫu đã chọn
    
    prompt = create_prompt(conversation)
    predicted_label = analyze_conversation(prompt)
    print(predicted_label)
    
    results.append({
        "file_name": f"output_{conversation['conversation_id']}.json",
        "predicted_label": predicted_label
    })

1103
Hài hước trong đoạn hội thoại này xuất phát từ sự không khớp giữa nội dung lời nói và ngữ cảnh. Khi một người đang nói về việc xem hài đọc thoại trên YouTube, một người khác đột ngột nhắc đến vấn đề trĩ nội trĩ ngoại, một chủ đề không liên quan và không phù hợp với ngữ cảnh. Điều này tạo ra một cảm giác hài hước vì sự ngẫu nhiên và không khớp giữa các lời nói.
5213
Hài hước trong đoạn hội thoại này xuất phát từ sự khác biệt giữa những gì người nói muốn nói và những gì người nghe hiểu. Người nói đang nói về sự tự do và quyền riêng tư, nhưng người nghe hiểu là đang được khuyến khích để làm những gì mình muốn, bao gồm cả việc mặc váy dài trên đường phố. Sự khác biệt này tạo ra một cảm giác hài hước khi người nghe phản ứng với sự ngạc nhiên và hào hứng.
3433
Hài hước trong đoạn hội thoại này xuất phát từ sự khác biệt giữa sự mong đợi ban đầu và kết quả thực tế. Khi người nói kể về việc mình đi cà phê với bà chị tâm linh, người nghe có thể mong đợi một câu chuyện về sự thiếu thấu cảm c

In [ ]:
Lưu lại kết quả dưới dạng file CSV

In [5]:
# Lưu kết quả vào file CSV
output_file = "D:\\DS310-P11-Nhom15\\results\\mistralai-Mixtral-8x22B-Instruct-v0.1-zero.csv"
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Kết quả đã được lưu vào {output_file}")

Kết quả đã được lưu vào D:\NLP\mistralai-Mixtral-8x22B-Instruct-v0.1-zero.csv
